# Generator of C lang samples (all of them I guess)

In [1]:
import numpy as np
import random
import math
from simple_c_v1_lang_generator import fill_vocabulary_c_v1, vocabulary_c_v1
from simple_c_v2_lang_generator import fill_vocabulary_c_v2, vocabulary_c_v2


## version 1.0

There is always type, an ID and openning curly brace at the beginning that simulates a function definition. Function can have parameters.


While generating I assume that if some ID is defined than it is defined "in order", it means that there should not be a situation when ID2 is defined before ID1 has been defined


### List of things that can be potentially included (TODO list)

* usage of pointers
* getting rid of STRINGs
* add floats
* add parenthesis in expressions
* choice frequency:
  * more definitions at the begginning

In [2]:
beg_call = "("
end_call = ")"
beg_block = "{"
end_block = "}"

built_in_types = ["int", "char"]
types_to_return_from_function = built_in_types + ["void"]

math_ops = ["-", "+", "/", "*"]
EMPTY = "EMPTY"
static_values_tokens = ["STRING", "NUM"]

IDs = []
funs = []
callable_funs = ["printf", "scanf"] + funs

fill_vocabulary_c_v1()
for index in range(16):
  current_id = "ID" + str(index)
  IDs.append(current_id)

print(len(vocabulary_c_v1))
print(vocabulary_c_v1)
print(IDs)
print(funs)

cv1Dict = {el:idx for idx,el in enumerate(vocabulary_c_v1)}

37
['void', 'int', 'char', 'STRING', 'NUM', 'return', 'printf', 'scanf', '-', '+', '/', '*', '&', '=', '(', ')', ';', '{', '}', 'EMPTY', ',', 'ID0', 'ID1', 'ID2', 'ID3', 'ID4', 'ID5', 'ID6', 'ID7', 'ID8', 'ID9', 'ID10', 'ID11', 'ID12', 'ID13', 'ID14', 'ID15']
['ID0', 'ID1', 'ID2', 'ID3', 'ID4', 'ID5', 'ID6', 'ID7', 'ID8', 'ID9', 'ID10', 'ID11', 'ID12', 'ID13', 'ID14', 'ID15']
[]


In [3]:
#
# defined_ids is a tuple -> (type, ID, definition_place)
# definition_place function-:
#   0: -body
#   1: -argument
#   2: -name

def val_token_dep_on_type(type):
  if type == "char":
    return "STRING"
  if type == "int":
    return "NUM"
  raise Exception(f"No known type '{type}'")

def get_first_free_ID(defined_ids):
  if len(defined_ids) >= len(IDs):
    raise Exception(f"hueheu, cannot give you another ID cause you already have {len(defined_ids)}")
  return IDs[len(defined_ids)]

def arg_pair(defined_ids):
  type = random.choice(built_in_types)
  return type, (type, get_first_free_ID(defined_ids), 1)

def gen_args(defined_ids, no_of_args):
  args = []
  ids = []
  for _ in range(no_of_args):
    type, id = arg_pair(defined_ids + ids)
    ids.append(id)
    args += [type, id[1]] + [","]

  # removing trailing comma (',')
  if len(args):
    del args[-1]

  return args, ids

def function_start():
  function_type = random.choice(types_to_return_from_function)
  to_return = [function_type, IDs[0], "("]
  defined_ids = [(function_type, IDs[0], 2)]
  no_of_args = random.randint(0, len(IDs) // (len(IDs) ** 0.5))
  args, ids = gen_args(defined_ids, no_of_args)
  
  defined_ids += ids
  to_return += args + [")", "{"]
  return to_return, defined_ids

# will add functions in some further generation
# returns list of ID that are passed as arguments
def call(defined_ids):
  to_return = []
  calls_choices = ["printf", "scanf"] # for now and this whole function will change when I add function calls (or just expression function to use here)
  chosen_call = random.choice(calls_choices)
  chosen_value = random.choice(static_values_tokens + [id[1] for id in defined_ids if id[2] != 2])
  to_return += [chosen_call, "(", "STRING", ",", chosen_value, ")", ";"]
  return to_return, []

assignment_definition_expression_max_length = 6
# can be called only if there are any IDs defined already (not as arguments)
# this function is probably going to be the most frequently changed part of this generation
def assignment(defined_ids):
  global assignment_definition_expression_max_length
  chosen_type, chosen_id, definition_place = random.choice(defined_ids)
  if definition_place == 0:
    return [], [] # cycling
  while(chosen_type == "void"):
    if len(defined_ids) == 1:
      return definition(defined_ids)
    chosen_type, chosen_id, definition_place = random.choice(defined_ids)
  return [chosen_id, "=", *(expression(defined_ids, chosen_type, [chosen_id], assignment_definition_expression_max_length)[0]), ";"], []

# returns ID that has been defined
def definition(defined_ids):
  global assignment_definition_expression_max_length
  if len(defined_ids) >= len(IDs):
    return [], [] # cycling
  id = get_first_free_ID(defined_ids)
  type = random.choice(built_in_types)
  if random.random() > 0.5: # no assignment
    tokens = [type, id, ";"]
  else:
    tokens = [type, id, "=", *(expression(defined_ids, type, [id], assignment_definition_expression_max_length)[0]), ";"]
  return tokens, [(type, id, 1)]


# the idea of this function is that it generates a sequence of tokens that will be used in other actions
def expression(defined_ids, of_type, excluded_ids, max_len, only_ids=False):
  length = random.randint(1, max_len)
  # if I would want to add recursion than should remove condition of excluding main function
  possible_vals = [id[1] for id in defined_ids if (id != "ID0" and id[0] == of_type and id[1] not in excluded_ids)] # ids
  if not only_ids or not possible_vals:
    if of_type == "int":
      val = "NUM"
    else:
      val = "STRING"
    possible_vals += [val]
  # print(possible_vals)
  operations = [random.choice(math_ops) for _ in range(length - 1)]

  # merging 
  to_return = []
  for op in operations:
    to_return += [random.choice(possible_vals), op]
  to_return.append(possible_vals[-1])
  return to_return, []

def calculate_choice_weights(defined_ids, counts):
  definition_w = (len(IDs) - len(defined_ids)) ** 1.1 / counts[1]
  call_w = len(callable_funs)
  # have fun with this static value
  # static * (all actions count - count of assignments) / num of counts
  assignment_w = float(3 * (sum(counts) - counts[2])) / len(counts)
  return [call_w, definition_w, assignment_w]


def gen_actions(defined_ids, no_of_actions, choice_weghts_calculator, actions):
  to_return = []
  new_defined_ids = []
  counts = [1 for _ in range(len(actions))]
  for _ in range(no_of_actions):
    weights = choice_weghts_calculator(defined_ids + new_defined_ids, counts)
    if len(actions) > 1:
      action = random.choices(range(len(actions)), weights)[0]
    else:
      action = 0
    # keep track of how many of them there is
    counts[action] += 1
    tokens, new_ids = actions[action](defined_ids + new_defined_ids)
    to_return += tokens
    new_defined_ids += new_ids
    if "return" in tokens:
      return to_return, new_defined_ids
    
  return to_return, new_defined_ids

def single_sample(choice_weghts_calculator, actions, no_of_actions):
  to_return = []
  tokens, defined_ids = function_start()
  to_return += tokens
  to_return += gen_actions(defined_ids, no_of_actions, choice_weghts_calculator, actions)[0]
  to_return += ["return", "NUM", ";", "}"] # end of function
  return to_return

def c_v1_simple_generate_samples(no_of_samples):
  no_of_actions = random.randint(8, 27)
  to_return = []
  choice_weghts_calculator = calculate_choice_weights
  actions = [call, definition, assignment]
  for _ in range(no_of_samples):
    to_return.append(single_sample(choice_weghts_calculator, actions, no_of_actions))
  return to_return


def fill_vector_with_empty_tokens(sample, length):
  if len(sample) > length:
    print("Sample is too long already. Yeeting it out of existence")
    return []
  for _ in range(length - len(sample)):
    sample.append("EMPTY")
  return sample

def tokens_to_vals(dict):
  def apply(sample):
    return [dict[token] for token in sample]
  return apply

In [4]:
generated = c_v1_simple_generate_samples(1)[0]
print(len(generated))
print(generated)
print(" ".join(generated).replace(";", ";\n").replace("{", "{\n"))
print(tokens_to_vals(cv1Dict)(generated))

82
['int', 'ID0', '(', ')', '{', 'char', 'ID1', ';', 'int', 'ID2', '=', 'NUM', '-', 'NUM', ';', 'int', 'ID3', ';', 'ID1', '=', 'STRING', '+', 'STRING', '/', 'STRING', '/', 'STRING', '+', 'STRING', ';', 'ID1', '=', 'STRING', '-', 'STRING', ';', 'int', 'ID4', '=', 'ID2', '/', 'NUM', '+', 'NUM', '/', 'NUM', ';', 'scanf', '(', 'STRING', ',', 'NUM', ')', ';', 'scanf', '(', 'STRING', ',', 'NUM', ')', ';', 'char', 'ID5', ';', 'ID3', '=', 'ID4', '/', 'ID4', '*', 'ID2', '-', 'ID2', '+', 'ID2', '-', 'NUM', ';', 'return', 'NUM', ';', '}']
int ID0 ( ) {
 char ID1 ;
 int ID2 = NUM - NUM ;
 int ID3 ;
 ID1 = STRING + STRING / STRING / STRING + STRING ;
 ID1 = STRING - STRING ;
 int ID4 = ID2 / NUM + NUM / NUM ;
 scanf ( STRING , NUM ) ;
 scanf ( STRING , NUM ) ;
 char ID5 ;
 ID3 = ID4 / ID4 * ID2 - ID2 + ID2 - NUM ;
 return NUM ;
 }
[1, 21, 14, 15, 17, 2, 22, 16, 1, 23, 13, 4, 8, 4, 16, 1, 24, 16, 22, 13, 3, 9, 3, 10, 3, 10, 3, 9, 3, 16, 22, 13, 3, 8, 3, 16, 1, 25, 13, 23, 10, 4, 9, 4, 10, 4, 16, 7, 

In [5]:
%%script false --no-raise-error
num_of_samples_to_generate = 10_000
dataset = c_v1_simple_generate_samples(num_of_samples_to_generate)
lens = [len(sample) for sample in dataset]
max_len, min_len, sum_len = max(lens), min(lens), sum(lens)
avg_lev = sum_len / len(dataset)

print(max_len)
print(min_len)
print(avg_lev)
target_length = 256
below_target_length = [l for l in lens if l < target_length]
print(len(below_target_length))
print(num_of_samples_to_generate - len(below_target_length))

In [6]:
%%script false --no-raise-error
converted_dataset_within_target_length = [tokens_to_vals(cv1Dict)(fill_vector_with_empty_tokens(sample, target_length)) for sample in dataset if len(sample) < target_length]
print(len(converted_dataset_within_target_length))
# fill samples to full length
final_dataset = np.asarray(converted_dataset_within_target_length)
print(final_dataset.shape)

### save dataset

In [7]:
%%script false --no-raise-error
data_dir_path = "..\\data\\simple_c_v1"
for idx, sample in enumerate(final_dataset):
  np.savetxt(f"{data_dir_path}\\sample_no{idx}.txt", sample, newline=" ", fmt="%u")

## version 2

There is always type, an ID and openning curly brace at the beginning that simulates a function definition. Function can have parameters.

While generating I assume that if some ID is defined than it is defined "in order", it means that there should not be a situation when ID2 is defined before ID1 has been defined

Additional tokens added in this version are if and else statements

In [35]:
def calculate_choice_weights_v2(defined_ids, counts):
  print(f"IDs len: {len(IDs)}")
  print(f"defined_ids len: {len(defined_ids)}")
  definition_w = (len(IDs) - len(defined_ids)) ** 1.1 / counts[1]
  call_w = len(callable_funs)
  # have fun with this static value
  # static * (all actions count - count of assignments) / num of counts
  assignment_w = float(3 * (sum(counts) - counts[2])) / len(counts)
  if_statement_w = float((sum(counts) - counts[2])) / len(counts)
  return [call_w, definition_w, assignment_w, if_statement_w]


compare_ops = [["<"], [">"], [">", "="], ["<", "="], ["=", "="], ["!", "="]]
def gen_single_condition(defined_ids):
  return expression(defined_ids, "int", [], 2, only_ids=True)[0] + random.choice(compare_ops) + ["NUM"]

logic_ops = [["|", "|"], ["&", "&"]]
def gen_conditions(defined_ids):
  to_return = []
  to_return += gen_single_condition(defined_ids)
  for _ in range(random.randint(0,2)):
    to_return += random.choice(logic_ops) + gen_single_condition(defined_ids)
  return to_return, []

def return_statement(defined_ids):
  return ["return", "NUM", ";"], []

def if_weights_choice_v2_calculator():
  def apply(defined_ids, counts):
    return calculate_choice_weights(defined_ids, counts) + [0.5]
  return apply

actions_for_if_c_v2 = [call, definition, assignment, return_statement]

consecutive_lvl = 0
def if_statement(defined_ids):
  global consecutive_lvl
  consecutive_lvl += 1
  to_return = ["if", "(", *(gen_conditions(defined_ids)[0]), ")", "{"]
  no_of_actions = random.randint(1, 4)
  to_return += gen_actions(defined_ids, no_of_actions, if_weights_choice_v2_calculator(), actions_for_if_c_v2)[0]
  to_return += ["}"]
  if random.random() < 0.5:
    # add else
    to_return += ["else"]
    if random.random() < 0.5 and consecutive_lvl < 3:
      # add if else
      to_return += if_statement(defined_ids)[0]
    else:
      to_return += ["{"]
      to_return += gen_actions(defined_ids, no_of_actions, if_weights_choice_v2_calculator(), actions_for_if_c_v2)[0]
      to_return += ["}"]
  consecutive_lvl -= 1
  return to_return, []

In [36]:
def calculate_choice_weights_v2_pure_action_block(defined_ids, counts):
  call_w = len(callable_funs)
  # have fun with this static value
  # static * (all actions count - count of assignments) / num of counts
  assignment_w = float(3 * (sum(counts) - counts[0])) / len(counts)
  return [call_w, assignment_w]

In [37]:
def single_sample_v2():
  on_action_weghts_calculator = lambda defined_ids, counts: [1.0]
  to_return = []
  defined_ids = []
  tokens, ids = function_start()
  to_return += tokens
  defined_ids += ids
  # gen definitions block
  tokens, ids = gen_actions(defined_ids, random.randint(2, 6), on_action_weghts_calculator, [definition])
  to_return += tokens
  defined_ids += ids
  # gen pure actions block
  tokens, ids = gen_actions(defined_ids, random.randint(2, 6), calculate_choice_weights_v2_pure_action_block, [assignment, call])
  to_return += tokens
  defined_ids += ids
  # gen if block
  tokens, ids = gen_actions(defined_ids, 1, on_action_weghts_calculator, [if_statement])
  to_return += tokens
  defined_ids += ids
  to_return += ["return", "NUM", ";", "}"] # end of function
  return to_return

In [38]:
def c_v2_simple_generate_samples(no_of_samples):
  to_return = []
  for _ in range(no_of_samples):
    to_return.append(single_sample_v2())
  return to_return

In [51]:
fill_vocabulary_c_v2()
print(len(vocabulary_c_v2))

assignment_definition_expression_max_length = 4
cv2Dict = {el:idx for idx,el in enumerate(vocabulary_c_v2)}

generated = c_v2_simple_generate_samples(1)[0]
print(len(generated))
print(generated)
print(" ".join(generated).replace(";", ";\n").replace("{", "{\n").replace("}","}\n"))
print(tokens_to_vals(cv2Dict)(generated))

Dictionary already filled
43
178
['int', 'ID0', '(', 'int', 'ID1', ')', '{', 'char', 'ID2', '=', 'STRING', '-', 'STRING', '/', 'STRING', ';', 'int', 'ID3', '=', 'ID1', '+', 'NUM', ';', 'ID2', '=', 'STRING', '*', 'STRING', ';', 'ID1', '=', 'NUM', ';', 'scanf', '(', 'STRING', ',', 'STRING', ')', ';', 'ID3', '=', 'NUM', ';', 'printf', '(', 'STRING', ',', 'ID3', ')', ';', 'if', '(', 'ID3', '<', '=', 'NUM', '|', '|', 'ID3', '-', 'ID3', '!', '=', 'NUM', ')', '{', 'int', 'ID4', '=', 'NUM', ';', 'ID2', '=', 'STRING', '+', 'STRING', '+', 'STRING', ';', '}', 'else', 'if', '(', 'ID3', '<', '=', 'NUM', ')', '{', 'int', 'ID4', '=', 'ID1', '*', 'ID1', '-', 'ID1', '-', 'NUM', ';', 'int', 'ID5', ';', 'ID3', '=', 'ID0', '+', 'NUM', ';', '}', 'else', 'if', '(', 'ID1', '*', 'ID3', '<', '=', 'NUM', '|', '|', 'ID3', '>', 'NUM', '|', '|', 'ID1', '+', 'ID3', '!', '=', 'NUM', ')', '{', 'char', 'ID4', '=', 'ID2', '+', 'STRING', '/', 'STRING', ';', 'char', 'ID5', '=', 'STRING', '+', 'ID2', '*', 'STRING', ';', '

In [52]:
num_of_samples_to_generate = 4096
dataset = c_v2_simple_generate_samples(num_of_samples_to_generate)
lens = [len(sample) for sample in dataset]
max_len, min_len, sum_len = max(lens), min(lens), sum(lens)
avg_lev = sum_len / len(dataset)

print(max_len)
print(min_len)
print(avg_lev)
target_length = 256
below_target_length = [l for l in lens if l < target_length]
print(len(below_target_length))
print(num_of_samples_to_generate - len(below_target_length))

249
48
110.387451171875
4096
0


In [53]:
converted_dataset_within_target_length = [tokens_to_vals(cv2Dict)(fill_vector_with_empty_tokens(sample, target_length)) for sample in dataset if len(sample) < target_length]
print(len(converted_dataset_within_target_length))
# fill samples to full length
final_dataset = np.asarray(converted_dataset_within_target_length)
print(final_dataset.shape)

4096
(4096, 256)


In [54]:
data_dir_path = "..\\data\\simple_c_v2"
for idx, sample in enumerate(final_dataset):
  np.savetxt(f"{data_dir_path}\\sample_no{idx}.txt", sample, newline=" ", fmt="%u")